# Trajectory Prediction Analysis

This notebook analyzes trajectory prediction model performance and visualizes results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from ml.models.trajectory_predictor import TrajectoryPredictor
from mpl_toolkits.mplot3d import Axes3D

## Load Trained Model

In [ ]:
# Load the trajectory prediction model
model = TrajectoryPredictor(input_size=6, hidden_size=256, sequence_length=100)
model.load_state_dict(torch.load('../trained_models/trajectory_predictor_v1.pth'))
model.eval()
print("Model loaded successfully!")

## Generate Test Trajectories

In [ ]:
# Generate a test orbital trajectory
def generate_orbital_trajectory(r0=10, v0=1.5, steps=200):
    """Generate a simple orbital trajectory for testing"""
    dt = 0.1
    G = 6.674e-11
    M = 5.97e24  # Earth mass
    
    positions = np.zeros((steps, 3))
    velocities = np.zeros((steps, 3))
    
    # Initial conditions
    positions[0] = [r0, 0, 0]
    velocities[0] = [0, v0, 0]
    
    for i in range(1, steps):
        r = np.linalg.norm(positions[i-1])
        a = -G * M / r**3 * positions[i-1]
        
        velocities[i] = velocities[i-1] + a * dt
        positions[i] = positions[i-1] + velocities[i] * dt
    
    return positions, velocities

true_pos, true_vel = generate_orbital_trajectory()
print(f"Generated trajectory with {len(true_pos)} points")

## Model Prediction

In [ ]:
# Use first 10 points to predict the rest
initial_sequence = 10
initial_conditions = np.concatenate([true_pos[:initial_sequence], true_vel[:initial_sequence]], axis=1)

# Predict trajectory
predicted_trajectory = model.predict_trajectory(
    initial_conditions, 
    prediction_steps=len(true_pos) - initial_sequence
)

print(f"Predicted trajectory shape: {predicted_trajectory.shape}")

## Visualization

In [ ]:
# 3D trajectory plot
fig = plt.figure(figsize=(15, 5))

# 3D trajectory comparison
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot(true_pos[:, 0], true_pos[:, 1], true_pos[:, 2], 'b-', label='True', linewidth=2)
ax1.plot(predicted_trajectory[:, 0], predicted_trajectory[:, 1], predicted_trajectory[:, 2], 
         'r--', label='Predicted', linewidth=2)
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.legend()
ax1.set_title('3D Trajectory Comparison')

# X-Y projection
ax2 = fig.add_subplot(132)
ax2.plot(true_pos[:, 0], true_pos[:, 1], 'b-', label='True', linewidth=2)
ax2.plot(predicted_trajectory[:, 0], predicted_trajectory[:, 1], 'r--', label='Predicted', linewidth=2)
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.legend()
ax2.set_title('X-Y Projection')
ax2.grid(True)

# Error over time
ax3 = fig.add_subplot(133)
errors = np.linalg.norm(true_pos[initial_sequence:] - predicted_trajectory, axis=1)
ax3.plot(errors)
ax3.set_xlabel('Time Step')
ax3.set_ylabel('Position Error')
ax3.set_title('Prediction Error Over Time')
ax3.grid(True)

plt.tight_layout()
plt.show()

print(f"Mean prediction error: {np.mean(errors):.4f}")
print(f"Max prediction error: {np.max(errors):.4f}")